In [1]:

import pandas as pd
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, SimpleRNN, LSTM, GRU, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax, Nadam, AdamW, Adagrad



2025-10-23 15:01:37.745550: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761231697.921787      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761231697.980199      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
# 1. Load data
df = pd.read_csv("/kaggle/input/linkedin-job-postings/postings.csv")
df.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,job_id,company_name,title,description,max_salary,pay_period,location,company_id,views,med_salary,...,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type,normalized_salary,zip_code,fips
0,921716,Corcoran Sawyer Smith,Marketing Coordinator,Job descriptionA leading real estate firm in N...,20.0,HOURLY,"Princeton, NJ",2774458.0,20.0,NaN,...,Requirements: \n\nWe are seeking a College or ...,1.713398e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,38480.0,8540.0,34021.0
1,1829192,NaN,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ...",50.0,HOURLY,"Fort Collins, CO",NaN,1.0,NaN,...,NaN,1.712858e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,83200.0,80521.0,8069.0
2,10998357,The National Exemplar,Assitant Restaurant Manager,The National Exemplar is accepting application...,65000.0,YEARLY,"Cincinnati, OH",64896719.0,8.0,NaN,...,We are currently accepting resumes for FOH - A...,1.713278e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,55000.0,45202.0,39061.0
3,23221523,"Abrams Fensterman, LLP",Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...,175000.0,YEARLY,"New Hyde Park, NY",766262.0,16.0,NaN,...,This position requires a baseline understandin...,1.712896e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,157500.0,11040.0,36059.0
4,35982263,NaN,Service Technician,Looking for HVAC service tech with experience ...,80000.0,YEARLY,"Burlington, IA",NaN,3.0,NaN,...,NaN,1.713452e+12,NaN,0,FULL_TIME,USD,BASE_SALARY,70000.0,52601.0,19057.0


In [3]:
df = df[['title', 'description']].dropna()
df.head()


,title,description
0,Marketing Coordinator,Job descriptionA leading real estate firm in N...
1,Mental Health Therapist/Counselor,"At Aspen Therapy and Wellness , we are committ..."
2,Assitant Restaurant Manager,The National Exemplar is accepting application...
3,Senior Elder Law / Trusts and Estates Associat...,Senior Associate Attorney - Elder Law / Trusts...
4,Service Technician,Looking for HVAC service tech with experience ...


In [4]:
# 2. Encode titles as classes
title_counts = df['title'].value_counts()
title_counts.unique()

array([673, 373, 354, 254, 238, 228, 211, 210, 204, 200, 195, 190, 189,
       181, 175, 171, 166, 162, 161, 155, 154, 146, 137, 135, 133, 130,
       127, 126, 123, 122, 119, 114, 113, 112, 110, 109, 107, 104, 103,
       102,  97,  95,  94,  92,  91,  90,  88,  87,  86,  84,  83,  81,
        79,  78,  76,  75,  74,  73,  72,  71,  70,  69,  68,  65,  64,
        63,  62,  61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,
        48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,  36,
        35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,  23,
        22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,  10,
         9,   8,   7,   6,   5,   4,   3,   2,   1])

In [5]:
job_title = df['title'].unique()
job_title

array(['Marketing Coordinator', 'Mental Health Therapist/Counselor',
       'Assitant Restaurant Manager', ...,
       'Staff Software Engineer, ML Serving Platform',
       'Account Executive, Oregon/Washington',
       'Marketing Social Media Specialist'], dtype=object)

In [6]:
titles = df['title'].astype(str)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
X_titles = vectorizer.fit_transform(titles)

In [8]:
#making the clusters to join the similar jobs together 
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X_titles)

In [9]:
# Step 3: Extract top terms per cluster to name them
terms = np.array(vectorizer.get_feature_names_out())
cluster_names = {}

In [10]:
for i in range(n_clusters):
    # get top 5 words with highest weights in that cluster center
    center_terms_idx = kmeans.cluster_centers_[i].argsort()[-5:][::-1]
    top_terms = terms[center_terms_idx]
    cluster_names[i] = " / ".join(top_terms)

In [ ]:
# Step 4: Add human-readable cluster names
df['job_category'] = df['cluster'].map(cluster_names)

In [12]:
# view results
for i, name in cluster_names.items():
    print(f"Cluster {i}: {name}")
    print(df[df['cluster']==i]['title'].head(5).to_list())
    print('-'*80)

Cluster 0: customer / service / representative / specialist / manager
['Inside Customer Service Associate', 'Customer Service / Reservationist', 'Customer Service Representative', 'Client Service Associate', 'Service Manager']
--------------------------------------------------------------------------------
Cluster 1: specialist / analyst / associate / senior / coordinator
['Marketing Coordinator', 'Mental Health Therapist/Counselor', 'Senior Elder Law / Trusts and Estates Associate Attorney', 'Economic Development and Planning Intern', 'Producer']
--------------------------------------------------------------------------------
Cluster 2: technician / maintenance / service / field / care
[' Service Technician', 'Service / Construction Technician', 'HVAC Technician', 'Robot Monitor & Maintenance Technician', 'Professional Window Cleaning Technician']
--------------------------------------------------------------------------------
Cluster 3: director / development / operations / sales / m

In [13]:
df = df[['description', 'job_category']].dropna()
df.head()

,description,job_category
0,Job descriptionA leading real estate firm in N...,specialist / analyst / associate / senior / co...
1,"At Aspen Therapy and Wellness , we are committ...",specialist / analyst / associate / senior / co...
2,The National Exemplar is accepting application...,manager / project / senior / account / product
3,Senior Associate Attorney - Elder Law / Trusts...,specialist / analyst / associate / senior / co...
4,Looking for HVAC service tech with experience ...,technician / maintenance / service / field / care


In [15]:
#standerize the txt like removing"Job descriptionA"
import re
def clean_text(text):
    text = text.lower()
    text = re.sub(r'job description[a-z]*', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['clean_text'] = df['description'].astype(str).apply(clean_text)

In [16]:
df.head()

,description,job_category,clean_text
0,Job descriptionA leading real estate firm in N...,specialist / analyst / associate / senior / co...,leading real estate firm in new jersey is seek...
1,"At Aspen Therapy and Wellness , we are committ...",specialist / analyst / associate / senior / co...,at aspen therapy and wellness we are committed...
2,The National Exemplar is accepting application...,manager / project / senior / account / product,the national exemplar is accepting application...
3,Senior Associate Attorney - Elder Law / Trusts...,specialist / analyst / associate / senior / co...,senior associate attorney elder law trusts and...
4,Looking for HVAC service tech with experience ...,technician / maintenance / service / field / care,looking for hvac service tech with experience ...


In [17]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['job_category'])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(
    df['clean_text'],
    df['label'],
    test_size=0.2,
    random_state=42,
    stratify=df['label']
)

In [19]:
max_words = 10000
max_len = 200
embedding_dim = 128

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(df["description"])

In [20]:
X = tokenizer.texts_to_sequences(df["description"])
X = pad_sequences(X, maxlen=max_len, padding='post', truncating='post')


In [21]:
# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["job_category"])

In [22]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
num_classes = len(label_encoder.classes_)
num_classes

10

In [24]:
#RNN Model
inputs_rnn = Input(shape=(max_len,), name="input_rnn")
x_rnn = Embedding(max_words, embedding_dim, input_length=max_len)(inputs_rnn)
hidden_rnn = SimpleRNN(128, activation='tanh')(x_rnn)
hidden_rnn = Dropout(0.3)(hidden_rnn)
outputs_rnn = Dense(num_classes, activation='softmax')(hidden_rnn)

model_rnn = Model(inputs=inputs_rnn, outputs=outputs_rnn, name="RNN_Model")

model_rnn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1761231919.928740      37 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Model: "RNN_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_rnn (InputLayer)          │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,314,186 (5.01 MB)

 Trainable params: 1,314,186 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
optimizer = Nadam(learning_rate=1e-3)
model_rnn.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [26]:
history_rnn = model_rnn.fit(X_train, y_train,validation_split=0.2,epochs=10,batch_size=64,verbose=1)

Epoch 1/10


I0000 00:00:1761231924.177596     119 service.cc:148] XLA service 0x7ba8300550d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1761231924.178061     119 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1761231924.437608     119 cuda_dnn.cc:529] Loaded cuDNN version 90300


  10/1239 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.2912 - loss: 2.1167

I0000 00:00:1761231925.784172     119 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1239/1239 ━━━━━━━━━━━━━━━━━━━━ 30s 21ms/step - accuracy: 0.5733 - loss: 1.5458 - val_accuracy: 0.6182 - val_loss: 1.2783
Epoch 2/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.6472 - loss: 1.1423 - val_accuracy: 0.6345 - val_loss: 1.1787
Epoch 3/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.7325 - loss: 0.8561 - val_accuracy: 0.6308 - val_loss: 1.2677
Epoch 4/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.7646 - loss: 0.7740 - val_accuracy: 0.6040 - val_loss: 1.4172
Epoch 5/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.8323 - loss: 0.5637 - val_accuracy: 0.6023 - val_loss: 1.4623
Epoch 6/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.8295 - loss: 0.5731 - val_accuracy: 0.5845 - val_loss: 1.5739
Epoch 7/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.8585 - loss: 0.4836 - val_accuracy: 0.5677 - val_loss: 1.7808
Epoch 8/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 24s 19ms/step - accuracy: 0.8813 - loss: 0.41

In [27]:
#LSTM Model
inputs_lstm = Input(shape=(max_len,), name="input_lstm")
x_lstm = Embedding(max_words, embedding_dim, input_length=max_len)(inputs_lstm)
hidden_lstm = LSTM(128, activation='tanh', kernel_initializer='glorot_uniform', dropout=0.5,recurrent_dropout=0.3)(x_lstm)
outputs_lstm = Dense(num_classes, activation='softmax',kernel_initializer='glorot_uniform')(hidden_lstm)
model_lstm = Model(inputs=inputs_lstm, outputs=outputs_lstm, name="LSTM_Model")
model_lstm.summary()

Model: "LSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_lstm (InputLayer)         │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_1 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,412,874 (5.39 MB)

 Trainable params: 1,412,874 (5.39 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
optimizer = Nadam(learning_rate=1e-3)
model_lstm.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [29]:
history_lstm = model_lstm.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

Epoch 1/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 662s 529ms/step - accuracy: 0.5697 - loss: 1.5207 - val_accuracy: 0.6255 - val_loss: 1.1040
Epoch 2/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 651s 526ms/step - accuracy: 0.6350 - loss: 1.0964 - val_accuracy: 0.6834 - val_loss: 0.9547
Epoch 3/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 648s 523ms/step - accuracy: 0.6954 - loss: 0.9031 - val_accuracy: 0.7712 - val_loss: 0.6927
Epoch 4/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 649s 524ms/step - accuracy: 0.7829 - loss: 0.6527 - val_accuracy: 0.8101 - val_loss: 0.5749
Epoch 5/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 650s 525ms/step - accuracy: 0.8246 - loss: 0.5321 - val_accuracy: 0.8260 - val_loss: 0.5338
Epoch 6/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 650s 524ms/step - accuracy: 0.8494 - loss: 0.4567 - val_accuracy: 0.8383 - val_loss: 0.5085
Epoch 7/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 646s 522ms/step - accuracy: 0.8715 - loss: 0.3952 - val_accuracy: 0.8431 - val_loss: 0.4945
Epoch 8/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 652s 526ms/step - ac

In [31]:
#Bidirectional LSTM Model
from tensorflow.keras.layers import Bidirectional
inputs_bilstm = Input(shape=(max_len,), name="input_bilstm")
x_bilstm = Embedding(max_words, embedding_dim, input_length=max_len)(inputs_bilstm)
hidden_bilstm = Bidirectional(LSTM(128, activation='tanh', kernel_initializer='glorot_uniform', dropout=0.5,recurrent_dropout=0.3))(x_bilstm)
hidden_bilstm = Dropout(0.5)(hidden_bilstm)
outputs_bilstm = Dense(num_classes, activation='softmax',kernel_initializer='glorot_uniform')(hidden_bilstm)
model_bilstm = Model(inputs=inputs_bilstm, outputs=outputs_bilstm, name="Bidirectional_LSTM_Model")
model_bilstm.summary()

Model: "Bidirectional_LSTM_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_bilstm (InputLayer)       │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_3 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 256)            │       263,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │         2,570 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,545,738 (5.90 MB)

 Trainable params: 1,545,738 (5.90 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
optimizer = Nadam(learning_rate=1e-3)
model_bilstm.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])

In [33]:
history_bilstm = model_bilstm.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

Epoch 1/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 985s 787ms/step - accuracy: 0.6068 - loss: 1.3946 - val_accuracy: 0.7439 - val_loss: 0.8552
Epoch 2/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 970s 783ms/step - accuracy: 0.7400 - loss: 0.8528 - val_accuracy: 0.7920 - val_loss: 0.6963
Epoch 3/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 971s 783ms/step - accuracy: 0.7785 - loss: 0.7232 - val_accuracy: 0.8046 - val_loss: 0.6285
Epoch 4/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 964s 778ms/step - accuracy: 0.7956 - loss: 0.6579 - val_accuracy: 0.8179 - val_loss: 0.5927
Epoch 5/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 969s 782ms/step - accuracy: 0.8145 - loss: 0.5801 - val_accuracy: 0.8231 - val_loss: 0.5716
Epoch 6/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 969s 782ms/step - accuracy: 0.8303 - loss: 0.5260 - val_accuracy: 0.8294 - val_loss: 0.5569
Epoch 7/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 979s 790ms/step - accuracy: 0.8488 - loss: 0.4708 - val_accuracy: 0.8368 - val_loss: 0.5417
Epoch 8/10
1239/1239 ━━━━━━━━━━━━━━━━━━━━ 978s 789ms/step - ac

In [34]:
#load GloVe embeddings
embedding_index = {}
with open("glove.6B.100d.txt", encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefficients = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefficients

print(f"Loaded {len(embedding_index)} word vectors.")

FileNotFoundError: [Errno 2] No such file or directory: 'glove.6B.100d.txt'

In [ ]:
#the embedding matrix
embedding_dim = 100
word_index = tokenizer.word_index
num_words = min(max_words, len(word_index) + 1)

embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector


In [ ]:
#bidirectional with glove embedinig
inputs_bilstm = Input(shape=(max_len,), name="input_bilstm_pretrained")

# Use pretrained embeddings (non-trainable to preserve knowledge)
embedding_layer = Embedding(
    input_dim=num_words,
    output_dim=embedding_dim,
    weights=[embedding_matrix],
    input_length=max_len,
    trainable=False
)(inputs_bilstm)

hidden_bilstm = Bidirectional(LSTM(128, activation='tanh', kernel_initializer='glorot_uniform', dropout=0.5,recurrent_dropout=0.3))(embedding_layer)
hidden_bilstm = Dropout(0.3)(hidden_bilstm)
outputs_bilstm = Dense(num_classes, activation='softmax',kernel_initializer='glorot_uniform')(hidden_bilstm)

model_bilstm_pretrained = Model(inputs=inputs_bilstm, outputs=outputs_bilstm, name="BiLSTM_Pretrained")

model_bilstm_pretrained.summary()

In [ ]:
optimizer = Nadam(learning_rate=1e-3)
model_bilstm_pretrained.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer, metrics=['accuracy'])


In [ ]:
history_bilstm_pretrained = model_bilstm_pretrained.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)


In [ ]:
# 4. GRU Model
inputs_gru = Input(shape=(max_len,), name="input_gru")
x_gru = Embedding(max_words, embedding_dim, input_length=max_len)(inputs_gru)
hidden_gru = GRU(128, activation='tanh', kernel_initializer='glorot_uniform', dropout=0.5,recurrent_dropout=0.3)(x_gru)
outputs_gru = Dense(num_classes, activation='softmax',kernel_initializer='glorot_uniform')(hidden_gru)
model_gru = Model(inputs=inputs_gru, outputs=outputs_gru, name="GRU_Model")
model_gru.summary()

In [ ]:
optimizer = Nadam(learning_rate=1e-3)
model_gru.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


In [ ]:
history_gru = model_gru.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=64,
    verbose=1
)

In [36]:
loss, accuracy = model_rnn.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

775/775 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.5580 - loss: 2.0128
Test loss: 2.0057
Test accuracy: 0.5594


In [37]:
loss, accuracy = model_lstm.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

775/775 ━━━━━━━━━━━━━━━━━━━━ 74s 96ms/step - accuracy: 0.8430 - loss: 0.5264
Test loss: 0.5354
Test accuracy: 0.8419


In [38]:
loss, accuracy = model_bilstm.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

775/775 ━━━━━━━━━━━━━━━━━━━━ 112s 144ms/step - accuracy: 0.8442 - loss: 0.5371
Test loss: 0.5501
Test accuracy: 0.8393


In [ ]:
loss, accuracy = model_bilstm_pretrained.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

In [ ]:
loss, accuracy = model_gru.evaluate(X_test, y_test)
print(f"Test loss: {loss:.4f}")
print(f"Test accuracy: {accuracy:.4f}")

In [39]:
model_rnn.save("RNN.keras")
model_lstm.save("LSTM.keras")
model_bilstm.save("Bidirectional_LSTM.keras")
#model_bilstm_pretrained.save("Embed_Bi_LSTM.keras")
#model_gru.save("GRU.keras")

In [40]:
import pickle

# Save tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Save label encoder
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [1]:
%%writefile jobs_final.py
import base64
import os
import streamlit as st
import numpy as np
import pandas as pd
from pathlib import Path
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Nadam
import pickle
import re
import logging
from io import BytesIO
import warnings
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# ------------------------------
# Basic config / logging
# ------------------------------
logging.basicConfig(level=logging.ERROR)
warnings.filterwarnings('ignore')

st.set_page_config(
    page_title='Job Title Predictor',
    layout='wide',
    initial_sidebar_state='auto'
)

# ------------------------------
# Background image setup
# ------------------------------
def set_background_image(image_path: str):
    if not Path(image_path).exists():
        st.warning(f"Background image not found: {image_path}")
        return

    with open(image_path, "rb") as f:
        encoded = f.read()
    b64 = base64.b64encode(encoded).decode()

    st.markdown(f"""
    <style>
    .stApp {{
        background-image: url("data:image/jpeg;base64,{b64}");
        background-size: cover;
        background-position: center;
        background-repeat: no-repeat;
        background-attachment: fixed;
    }}
    .stApp::before {{
        content: "";
        position: fixed;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        background-color: rgba(0, 0, 0, 0);  /* ← Adjust this value */
        z-index: -1;
}}
    .app-backdrop {{
        background: rgba(255,255,255,0.85);
        padding: 12px;
        border-radius: 8px;
        position: relative;
        z-index: 10;
    }}
    </style>
    """, unsafe_allow_html=True)

# Helpers: load artifacts & preprocessing
try:
    cache_resource = st.cache_resource
except Exception:
    cache_resource = st.cache(allow_output_mutation=True)

@cache_resource
def load_artifacts():
    with open('tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    with open('label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    model = load_model('LSTM.keras', compile=False)
    optimizer = Nadam(learning_rate=1e-3)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return tokenizer, label_encoder, model

def clean_text(text):
    if not isinstance(text, str): return ''
    text = text.lower()
    text = re.sub(r'job description[a-z]*', '', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def predict_top_k(model, tokenizer, label_encoder, text, max_len=200, k=3):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    probs = model.predict(padded, verbose=0)[0]
    top_idx = probs.argsort()[::-1][:k]
    labels = label_encoder.inverse_transform(top_idx)
    scores = probs[top_idx]
    return list(labels), list(scores)

def create_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')
    return fig

def main():
    set_background_image("pexels-ruslan-burlaka-40570-140945.jpg")

    st.markdown('<div class="app-backdrop">', unsafe_allow_html=True)
    st.title('Job Title Prediction from Description')
    st.write('Paste a job description and the model will predict the most likely job title / category.')
    st.markdown('</div>', unsafe_allow_html=True)

    with st.spinner('Loading model and artifacts...'):
        try:
            tokenizer, label_encoder, model = load_artifacts()
        except Exception:
            st.error("Failed loading artifacts. Ensure tokenizer.pkl, label_encoder.pkl, and LSTM.keras exist.")
            return

    st.sidebar.header('Options & Info')
    st.sidebar.write(f'Model: LSTM.keras')
    st.sidebar.write(f'Classes: {len(label_encoder.classes_)}')
    top_k = st.sidebar.slider('Show top K predictions', min_value=1, max_value=10, value=3)

    st.sidebar.markdown('---')
    st.sidebar.markdown('Examples:')
    example = st.sidebar.selectbox('Pick a sample description', [
        '',
        'We are looking for a software engineer with experience in Python, TensorFlow, and REST APIs.',
        'Seeking a marketing manager to lead product campaigns and manage social media.',
        'Experienced data scientist required: SQL, Python, machine learning models, production deployment.',
        'Administrative assistant needed for scheduling, record keeping, and customer support.'
    ])

    st.markdown('<div class="app-backdrop" style="margin-top:15px;">', unsafe_allow_html=True)
    description = st.text_area('Paste your job description here:', value=example, height=220)

    col_left, col_right = st.columns([3,1])
    with col_right:
        if st.button('Clear'):
            description = ''

    if st.button('Predict Job Title'):
        if not description or len(description.strip()) < 10:
            st.warning('Please paste a longer job description (at least ~10 characters).')
        else:
            cleaned = clean_text(description)
            labels, scores = predict_top_k(model, tokenizer, label_encoder, cleaned, max_len=200, k=top_k)

            res_col_left, res_col_right = st.columns(2)
            with res_col_left:
                st.markdown('<div class="prediction-box">', unsafe_allow_html=True)
                if labels:
                    st.success(f'Primary prediction: {labels[0]} — {scores[0]*100:.2f}% confidence')
                else:
                    st.error("No prediction returned.")
                st.markdown('</div>', unsafe_allow_html=True)

                st.subheader('Top predictions')
                results = pd.DataFrame({
                    'Job Category': labels,
                    'Confidence': [f'{s*100:.2f}%' for s in scores]
                })
                st.table(results)

            with res_col_right:
                st.subheader('Confidence Distribution')
                if labels and scores:
                    fig = px.pie(
                        values=[s * 100 for s in scores],
                        names=labels,
                        title='Prediction Confidence Distribution',
                        hole=0.4
                    )
                    st.plotly_chart(fig, use_container_width=True)

            st.subheader('Key Terms Visualization')
            wc_fig = create_wordcloud(cleaned)
            if wc_fig:
                st.pyplot(wc_fig)

            with st.expander("View Processed Text"):
                st.subheader('Description (cleaned)')
                st.write(cleaned)
                st.subheader('Original description')
                st.write(description)

            results_df = pd.DataFrame({
                'label': labels,
                'confidence': [float(s) for s in scores],
                'confidence_pct': [f'{s*100:.2f}%' for s in scores]
            })
            csv_buf = BytesIO()
            results_df.to_csv(csv_buf, index=False)
            csv_buf.seek(0)
            st.download_button(
                label='Download Results (CSV)',
                data=csv_buf,
                file_name='job_predictions.csv',
                mime='text/csv'
            )

    st.markdown('</div>', unsafe_allow_html=True)

    st.markdown('---')
    st.markdown(
        '<div class="small">Note: Predictions are based on a trained model and may not be perfect. '
        'Consider this as an assistive suggestion.</div>',
        unsafe_allow_html=True
    )

if __name__ == '__main__':
    main()


Overwriting jobs_final.py


In [1]:
#updated
%%writefile jobs_final.py
import base64
import os
import streamlit as st
import numpy as np
import pandas as pd
from pathlib import Path
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Nadam
import pickle
import re
import logging
from io import BytesIO
import warnings
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt


logging.basicConfig(level=logging.ERROR)
warnings.filterwarnings('ignore')

st.set_page_config(
    page_title='Job Title Predictor',
    layout='wide',
    initial_sidebar_state='auto'
)


def set_background_image(image_path: str):
    if not Path(image_path).exists():
        st.warning(f"Background image not found: {image_path}")
        return
    with open(image_path, "rb") as f:
        encoded = f.read()
    b64 = base64.b64encode(encoded).decode()
    st.markdown(f"""
    <style>
    .stApp {{
        background-image: url("data:image/jpeg;base64,{b64}");
        background-size: cover;
        background-position: center;
        background-repeat: no-repeat;
        background-attachment: fixed;
    }}
    .stApp::before {{
        content: "";
        position: fixed;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        background-color: rgba(0, 0, 0, 0);
        z-index: -1;
    }}
    .app-backdrop {{
        background: rgba(255,255,255,0.85);
        padding: 12px;
        border-radius: 8px;
        position: relative;
        z-index: 10;
    }}
    </style>
    """, unsafe_allow_html=True)

try:
    cache_resource = st.cache_resource
except Exception:
    cache_resource = st.cache(allow_output_mutation=True)

@cache_resource
def load_artifacts(model_name='LSTM.keras'):
    with open('tokenizer.pkl', 'rb') as f:
        tokenizer = pickle.load(f)
    with open('label_encoder.pkl', 'rb') as f:
        label_encoder = pickle.load(f)
    model_path = os.path.join('models', model_name)
    model = load_model(model_path, compile=False)
    optimizer = Nadam(learning_rate=1e-3)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return tokenizer, label_encoder, model

def clean_text(text):
    if not isinstance(text, str): return ''
    text = text.lower()
    text = re.sub(r'job description[a-z]*', '', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def predict_top_k(model, tokenizer, label_encoder, text, max_len=200, k=3):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post', truncating='post')
    probs = model.predict(padded, verbose=0)[0]
    top_idx = probs.argsort()[::-1][:k]
    labels = label_encoder.inverse_transform(top_idx)
    scores = probs[top_idx]
    return list(labels), list(scores)

def create_wordcloud(text):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.imshow(wordcloud, interpolation='bilinear')
    ax.axis('off')
    return fig

def main():
    set_background_image("pexels-ruslan-burlaka-40570-140945.jpg")

    st.markdown('<div class="app-backdrop">', unsafe_allow_html=True)
    st.title('Job Title Prediction from Description')
    st.write('Paste a job description and the model will predict the most likely job title / category.')
    st.markdown('</div>', unsafe_allow_html=True)

    # Sidebar: model selector
    st.sidebar.header('Options & Info')
    model_choices = sorted([f for f in os.listdir('models') if f.endswith('.keras')])
    selected_model = st.sidebar.selectbox('Choose model', model_choices)

    with st.spinner(f'Loading model: {selected_model}'):
        try:
            tokenizer, label_encoder, model = load_artifacts(selected_model)
        except Exception:
            st.error(f"Failed loading model: {selected_model}")
            return

    st.sidebar.write(f'Model: {selected_model}')
    st.sidebar.write(f'Classes: {len(label_encoder.classes_)}')
    top_k = st.sidebar.slider('Show top K predictions', min_value=1, max_value=10, value=3)

    st.sidebar.markdown('---')
    st.sidebar.markdown('Examples:')
    example = st.sidebar.selectbox('Pick a sample description', [
        '',
        'We are looking for a software engineer with experience in Python, TensorFlow, and REST APIs.',
        'Seeking a marketing manager to lead product campaigns and manage social media.',
        'Experienced data scientist required: SQL, Python, machine learning models, production deployment.',
        'Administrative assistant needed for scheduling, record keeping, and customer support.'
    ])

    st.markdown('<div class="app-backdrop" style="margin-top:15px;">', unsafe_allow_html=True)
    description = st.text_area('Paste your job description here:', value=example, height=220)

    col_left, col_right = st.columns([3,1])
    with col_right:
        if st.button('Clear'):
            description = ''

    if st.button('Predict Job Title'):
        if not description or len(description.strip()) < 10:
            st.warning('Please paste a longer job description (at least ~10 characters).')
        else:
            cleaned = clean_text(description)
            labels, scores = predict_top_k(model, tokenizer, label_encoder, cleaned, max_len=200, k=top_k)

            res_col_left, res_col_right = st.columns(2)
            with res_col_left:
                st.markdown('<div class="prediction-box">', unsafe_allow_html=True)
                if labels:
                    st.success(f'Primary prediction: {labels[0]} — {scores[0]*100:.2f}% confidence')
                else:
                    st.error("No prediction returned.")
                st.markdown('</div>', unsafe_allow_html=True)

                st.subheader('Top predictions')
                results = pd.DataFrame({
                    'Job Category': labels,
                    'Confidence': [f'{s*100:.2f}%' for s in scores]
                })
                st.table(results)

            with res_col_right:
                st.subheader('Confidence Distribution')
                if labels and scores:
                    fig = px.pie(
                        values=[s * 100 for s in scores],
                        names=labels,
                        title='Prediction Confidence Distribution',
                        hole=0.4
                    )
                    st.plotly_chart(fig, use_container_width=True)

            st.subheader('Key Terms Visualization')
            wc_fig = create_wordcloud(cleaned)
            if wc_fig:
                st.pyplot(wc_fig)

            with st.expander("View Processed Text"):
                st.subheader('Description (cleaned)')
                st.write(cleaned)
                st.subheader('Original description')
                st.write(description)

            results_df = pd.DataFrame({
                'label': labels,
                'confidence': [float(s) for s in scores],
                'confidence_pct': [f'{s*100:.2f}%' for s in scores]
            })
            csv_buf = BytesIO()
            results_df.to_csv(csv_buf, index=False)
            csv_buf.seek(0)
            st.download_button(
                label='Download Results (CSV)',
                data=csv_buf,
                file_name='job_predictions.csv',
                mime='text/csv'
            )

    st.markdown('</div>', unsafe_allow_html=True)

    st.markdown('---')
    st.markdown(
        '<div class="small">Note: Predictions are based on a trained model and may not be perfect. '
        'Consider this as an assistive suggestion.</div>',
        unsafe_allow_html=True
    )

if __name__ == '__main__':
    main()


UsageError: Line magic function `%%writefile` not found.


In [ ]:
streamlit run jobs_final.py

In [2]:
! pipreqs ./

INFO: Successfully saved requirements file in ./requirements.txt
